In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
#데이터셋 불러오기
df_org = pd.read_csv('/content/drive/MyDrive/steam.csv').iloc[0:10000]

df_org

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,501830,The Orb Chambers II,2016-08-01,1,StarSystemStudios™;Jaryd Brad Spann,StarSystemStudios™,windows,0,Single-player;Steam Achievements;Steam Trading...,Action;Casual;Indie;Strategy,Strategy;Action;Indie,35,40,44,278,288,50000-100000,0.79
9996,501840,Don't Disturb,2016-09-01,1,Midnight Party,PQube Limited,windows;mac;linux,0,Single-player;Steam Trading Cards,Indie,Indie;Singleplayer;Puzzle,0,254,48,0,0,0-20000,2.69
9997,501850,DCR: Drive.Crash.Repeat,2018-08-21,1,Tarboosh Games,Tarboosh Games,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Casual;Indie;Racing,Indie;Casual;Adventure,37,5,0,0,0,0-20000,6.19
9998,501860,Slayer Shock,2016-09-29,1,Minor Key Games,Minor Key Games,windows;mac;linux,0,Single-player;Steam Achievements;Full controll...,Action;Indie;RPG;Strategy,Action;Indie;RPG,15,39,24,0,0,0-20000,14.99


In [7]:
df_org.columns

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')

In [8]:
# Content-based filtering에 도움이 될 열만 남기기
df = df_org[['appid', 'name', 'english', 'developer', 'publisher', 'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags', 'achievements', 'owners', 'price']]

df

,appid,name,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,owners,price
0,10,Counter-Strike,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,10000000-20000000,7.19
1,20,Team Fortress Classic,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,5000000-10000000,3.99
2,30,Day of Defeat,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,5000000-10000000,3.99
3,40,Deathmatch Classic,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,501830,The Orb Chambers II,1,StarSystemStudios™;Jaryd Brad Spann,StarSystemStudios™,windows,0,Single-player;Steam Achievements;Steam Trading...,Action;Casual;Indie;Strategy,Strategy;Action;Indie,35,50000-100000,0.79
9996,501840,Don't Disturb,1,Midnight Party,PQube Limited,windows;mac;linux,0,Single-player;Steam Trading Cards,Indie,Indie;Singleplayer;Puzzle,0,0-20000,2.69
9997,501850,DCR: Drive.Crash.Repeat,1,Tarboosh Games,Tarboosh Games,windows;mac,0,Single-player;Steam Achievements;Steam Trading...,Adventure;Casual;Indie;Racing,Indie;Casual;Adventure,37,0-20000,6.19
9998,501860,Slayer Shock,1,Minor Key Games,Minor Key Games,windows;mac;linux,0,Single-player;Steam Achievements;Full controll...,Action;Indie;RPG;Strategy,Action;Indie;RPG,15,0-20000,14.99


In [9]:
#데이터 전처리
columns = ['platforms', 'categories', 'genres', 'steamspy_tags']
unique_values = {}

for col in columns:
    df[col] = df[col].str.split(';')

df

<ipython-input-9-ed603f0a4271>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.split(';')


,appid,name,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,owners,price
0,10,Counter-Strike,1,Valve,Valve,"[windows, mac, linux]",0,"[Multi-player, Online Multi-Player, Local Mult...",[Action],"[Action, FPS, Multiplayer]",0,10000000-20000000,7.19
1,20,Team Fortress Classic,1,Valve,Valve,"[windows, mac, linux]",0,"[Multi-player, Online Multi-Player, Local Mult...",[Action],"[Action, FPS, Multiplayer]",0,5000000-10000000,3.99
2,30,Day of Defeat,1,Valve,Valve,"[windows, mac, linux]",0,"[Multi-player, Valve Anti-Cheat enabled]",[Action],"[FPS, World War II, Multiplayer]",0,5000000-10000000,3.99
3,40,Deathmatch Classic,1,Valve,Valve,"[windows, mac, linux]",0,"[Multi-player, Online Multi-Player, Local Mult...",[Action],"[Action, FPS, Multiplayer]",0,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1,Gearbox Software,Valve,"[windows, mac, linux]",0,"[Single-player, Multi-player, Valve Anti-Cheat...",[Action],"[FPS, Action, Sci-fi]",0,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,501830,The Orb Chambers II,1,StarSystemStudios™;Jaryd Brad Spann,StarSystemStudios™,[windows],0,"[Single-player, Steam Achievements, Steam Trad...","[Action, Casual, Indie, Strategy]","[Strategy, Action, Indie]",35,50000-100000,0.79
9996,501840,Don't Disturb,1,Midnight Party,PQube Limited,"[windows, mac, linux]",0,"[Single-player, Steam Trading Cards]",[Indie],"[Indie, Singleplayer, Puzzle]",0,0-20000,2.69
9997,501850,DCR: Drive.Crash.Repeat,1,Tarboosh Games,Tarboosh Games,"[windows, mac]",0,"[Single-player, Steam Achievements, Steam Trad...","[Adventure, Casual, Indie, Racing]","[Indie, Casual, Adventure]",37,0-20000,6.19
9998,501860,Slayer Shock,1,Minor Key Games,Minor Key Games,"[windows, mac, linux]",0,"[Single-player, Steam Achievements, Full contr...","[Action, Indie, RPG, Strategy]","[Action, Indie, RPG]",15,0-20000,14.99


In [10]:
# Embedding (Countventorizer)
df_copy = df.copy().drop(columns=['appid', 'name'])
content = df_copy.apply(lambda row: ' '.join(map(str, row)), axis=1)

vectorizer = CountVectorizer()
features = vectorizer.fit_transform(content)

print(features)

  (0, 7880)	3
  (0, 8196)	1
  (0, 4593)	1
  (0, 4445)	1
  (0, 5097)	3
  (0, 5842)	3
  (0, 5481)	1
  (0, 4480)	1
  (0, 578)	1
  (0, 1459)	1
  (0, 2461)	1
  (0, 363)	2
  (0, 2852)	1
  (0, 5099)	1
  (0, 9)	1
  (0, 112)	1
  (0, 101)	1
  (1, 7880)	3
  (1, 8196)	1
  (1, 4593)	1
  (1, 4445)	1
  (1, 5097)	3
  (1, 5842)	3
  (1, 5481)	1
  (1, 4480)	1
  :	:
  (9998, 69)	1
  (9998, 2926)	1
  (9998, 7236)	1
  (9998, 3693)	2
  (9998, 109)	1
  (9998, 60)	1
  (9998, 6443)	2
  (9998, 3016)	2
  (9998, 4081)	2
  (9998, 4923)	2
  (9999, 8196)	1
  (9999, 5842)	1
  (9999, 363)	2
  (9999, 101)	1
  (9999, 6836)	1
  (9999, 7175)	1
  (9999, 7677)	1
  (9999, 1360)	1
  (9999, 3693)	2
  (9999, 241)	1
  (9999, 7)	1
  (9999, 6443)	2
  (9999, 3016)	2
  (9999, 7068)	1
  (9999, 2485)	1


In [13]:
# user_profile은 item_profile 중 하나를 가져와서 씀
# 유사도 비교

sim = cosine_similarity(features, features).argsort()[:, ::-1]
print(sim)

[[1.         0.95454545 0.76870611 ... 0.13957263 0.22990024 0.26111648]
 [0.95454545 1.         0.82783735 ... 0.11631053 0.25289027 0.23210354]
 [0.76870611 0.82783735 1.         ... 0.09078413 0.20935201 0.15097027]
 ...
 [0.13957263 0.11631053 0.09078413 ... 1.         0.47062081 0.50482679]
 [0.22990024 0.25289027 0.20935201 ... 0.47062081 1.         0.61631563]
 [0.26111648 0.23210354 0.15097027 ... 0.50482679 0.61631563 1.        ]]
100000000
[[   0    1    3 ... 4770 8353 5075]
 [   1    3    5 ... 5586 8353 5075]
 [   2   15    5 ... 5586 8353 5075]
 ...
 [9997 8305 6387 ... 2924 1263 8353]
 [9998 4162 2021 ... 2699 6389 1438]
 [9999 9930 9165 ... 4318 7635 8125]]
100000000


In [17]:
appid = 10
rec_cnt = 5

target_game_idx = df[df['appid'] == appid].index.values

sim_idx = sim[target_game_idx, :rec_cnt+1].reshape(-1)

# remove index of itself
sim_idx = sim_idx[sim_idx != target_game_idx]

# make the name list of top 10 recommended games
recommended = df.iloc[sim_idx][:10]
result = recommended['name'].to_list()

print(result)

['Team Fortress Classic', 'Deathmatch Classic', 'Ricochet', 'Counter-Strike: Condition Zero', 'Half-Life']
